<a href="https://colab.research.google.com/github/zalfajhra/2025_VK_TI2C/blob/main/Jobsheet03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>